In [1]:
import torch
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from functools import partial
import torch
from lib.timer import Timer
from lib.utils import load_obj, natural_key
from datasets.indoor import IndoorDataset
from datasets.modelnet import get_train_datasets, get_test_datasets
import os,re,sys,json,yaml,random, argparse, torch, pickle
from easydict import EasyDict as edict
from configs.models import architectures
from models.architectures import KPFCNN
import open3d as o3d
import matplotlib.pyplot as plt
# import open3d.visualization.jupyter as o3d_jupyter
import plotly.graph_objects as go


def load_config(path):
    """
    Loads config file:

    Args:
        path (str): path to the config file

    Returns: 
        config (dict): dictionary of the configuration parameters, merge sub_dicts

    """
    with open(path,'r') as f:
        cfg = yaml.safe_load(f)
    
    config = dict()
    for key, value in cfg.items():
        for k,v in value.items():
            config[k] = v

    return config




def get_datasets(config):
    info_train = load_obj(config.train_info)
    train_set = IndoorDataset(info_train,config,data_augmentation=True)
    return train_set

# def get_colored_image_pca_sep(feature, name):
#     import matplotlib.pyplot as plt
#     # Reshape the features to [num_samples, num_features]
#     w, h, d = feature.shape
#     reshaped_features = feature.reshape((w * h, d))

#     # Apply PCA to reduce dimensionality to 3
#     pca = PCA(n_components=3)
#     pca_result = pca.fit_transform(reshaped_features)

#     # Normalize the PCA results to 0-1 range for visualization
#     pca_result -= pca_result.min(axis=0)
#     pca_result /= pca_result.max(axis=0)

#     # Reshape back to the original image shape
#     image_data = pca_result.reshape((w, h, 3))

#     # Display and save the image
#     plt.imshow(image_data)
#     plt.axis('off')
#     plt.savefig(f'img_{name}.jpg', bbox_inches='tight', pad_inches=0)


# def get_colored_point_cloud_from_soft_labels(xyz, soft_labels, name):
#     # Convert soft labels to hard labels
#     hard_labels = np.argmax(soft_labels, axis=1)
#     unique_labels = np.unique(hard_labels)
#     # Generate a colormap with 21 distinct colors
#     cmap = plt.get_cmap('tab20', len(unique_labels))  # 'tab20b' has 20 distinct colors, adjust as needed for 21
#     # Map hard labels to colors using the colormap
#     colors = np.array([cmap(i)[:3] for i in hard_labels])  # Extract RGB components
#     # Create and color the point cloud
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(xyz)
#     pcd.colors = o3d.utility.Vector3dVector(colors)
#     o3d.visualization.draw_geometries([pcd])

#     # Save the point cloud
#     o3d.io.write_point_cloud(name + f'.ply', pcd)


# def creat_labeled_point_cloud(points, labels, name):
#     """
#     Creates a point cloud where each point is colored based on its label, and saves it to a .ply file.

#     Parameters:
#     - points: NumPy array of shape (N, 3) representing the point cloud.
#     - labels: NumPy array of shape (N,) containing integer labels for each point.
#     - name: String representing the base filename for the output .ply file.
#     """
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(points)

#     # Map labels to colors using the predefined color map
#     # Normalize RGB values to [0, 1] range as expected by Open3D
#     colors = np.array([SCANNET_COLOR_MAP_200.get(label+1, (0., 0., 0.)) for label in labels]) / 255.0
#     pcd.colors = o3d.utility.Vector3dVector(colors)
#     # o3d.visualization.draw_geometries([pcd])

#     # Save the colored point cloud to a .ply file
#     o3d.io.write_point_cloud(f'{name}.ply', pcd)


# def get_colored_point_cloud_pca_sep(xyz, feature, name=None):
#     """N x D"""
#     pca = PCA(n_components=3)
#     pca_gf = pca.fit_transform(feature)
#     pca_gf = (pca_gf + np.abs(pca_gf.min(0))) / (pca_gf.ptp(0) + 1e-4)
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(xyz[:, 0:3])
#     pcd.colors = o3d.utility.Vector3dVector(pca_gf)
#     o3d.visualization.draw_geometries([pcd])
#     # o3d.io.write_point_cloud(name + f'.ply', pcd)


# def visualize_clusters(point_cloud, labels, name=None):
#     # Generate a color map where each cluster has a unique color
#     colors = np.array([SCANNET_COLOR_MAP_200.get(label, (255.0, 255.0, 255.0)) for label in labels]) / 255.0

#     # Create an Open3D point cloud object
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(point_cloud)
#     pcd.colors = o3d.utility.Vector3dVector(colors)

#     # Visualize the point cloud
#     # o3d.visualization.draw_geometries([pcd])
#     o3d.io.write_point_cloud(name + '.ply', pcd)



# def vis_points(points):
#     # Convert PyTorch tensor to NumPy array
#     point_cloud_np = points.numpy()

#     # Create Open3D PointCloud object
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(point_cloud_np)

#     # Visualize the point cloud
#     o3d.visualization.draw_geometries([pcd])

#     # Optional: If you want to add colors to the points, you can do so as follows:
#     # Create random colors for each point
#     # colors = torch.rand(point_cloud_np.shape[0], 3).numpy()
#     # pcd.colors = o3d.utility.Vector3dVector(colors)
#     #
#     # # Visualize the colored point cloud
#     # o3d.visualization.draw_geometries([pcd])

if __name__ == '__main__':
    config = edict(load_config('configs/train/indoor.yaml'))
    # print(config)
    train_set = get_datasets(config)
    # print(train_set[500])
    point = train_set[10]
    x=torch.zeros_like(torch.from_numpy(point['src_pcd'])).cuda().float()
    color1 = point['src_color1'].cuda()
    src1_inds2d=point['src1_inds2d'].cuda()
    src1_inds3d=point['src1_inds3d'].cuda()
    print(color1.shape)

    src1_feats = color1[:, src1_inds2d[:, 1], src1_inds2d[:, 0]].transpose(1, 0)
    # src1_point_with_2dfeature = torch.cat(
    #                 (src1_feats.transpose(1, 0), torch.ones(src1_feats.shape[1], 1).cuda()), dim=-1).detach()
    x[src1_inds3d, :] = src1_feats.float()
    # print(x)
    # 0/0
    # 创建源点云
    src_pcd = point['src_pcd']
    # 创建目标点云
    tgt_pcd = point['tgt_pcd']

    # 创建 Open3D 点云对象
    pcd = o3d.geometry.PointCloud()

    # 设置点云坐标
    pcd.points = o3d.utility.Vector3dVector(src_pcd)

    # 设置颜色
    pcd.colors = o3d.utility.Vector3dVector(x.cpu().numpy())

    # 保存为 PLY 文件
    o3d.io.write_point_cloud('output_mask.ply', pcd)

    masks_loaded = np.load('masks.npy')
    print(masks_loaded.shape)
    print(masks_loaded[0])
    print(masks_loaded[0].shape)

    bool_array = np.squeeze(masks_loaded[0])  # 变为 (480, 640)

    # 创建一个形状为 (3, 480, 640) 的 RGB 图像数组
    color_image = np.zeros((3, bool_array.shape[0],  bool_array.shape[1]))

    # 将 True 映射为红色，False 映射为蓝色
    color_image[0, bool_array] = 1  # 红色通道
    color_image[1, ~bool_array] = 0  # 绿色通道（保持为0）
    color_image[2, bool_array] = 0  # 蓝色通道（保持为0）
    color_image[0, ~bool_array] = 0  # 红色通道（保持为0）
    color_image[1, bool_array] = 0  # 绿色通道（保持为0）
    color_image[2, ~bool_array] = 1  # 蓝色通道

    print(color_image)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/cx/anaconda3/envs/PCR-CG/lib/python3.9/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


item_10_bundlefusion-apt0__src4000__tgt4400
torch.Size([3, 120, 160])
(8, 1, 480, 640)
[[[False False False ...  True  True  True]
  [False False False ...  True  True  True]
  [False False False ...  True  True  True]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]]
(1, 480, 640)
[[[0. 0. 0. ... 1. 1. 1.]
  [0. 0. 0. ... 1. 1. 1.]
  [0. 0. 0. ... 1. 1. 1.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 1. 1. ... 0. 0. 0.]
  [1. 1. 1. ... 0. 0. 0.]
  [1. 1. 1. ... 0. 0. 0.]
  ...
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]
  [1. 1. 1. ... 1. 1. 1.]]]
